# Complete Project Setup with GitHub Integration

This notebook does everything in one go:
1. Creates project structure
2. Sets up all modules and configs
3. Pushes to GitHub

Run all cells in sequence.

## Prerequisites

Before running this notebook:

1. Create GitHub repository at https://github.com/new
   - Name: `financial-ml-system`
   - Set to **Public**
   - Do NOT initialize with README

2. Create Personal Access Token at https://github.com/settings/tokens
   - Click: Generate new token (classic)
   - Select scope: **repo** (full control)
   - Copy the token

## Part 1: Install Dependencies

In [ ]:
%%capture
!pip install yfinance pandas numpy scikit-learn tensorflow matplotlib seaborn plotly pyyaml joblib

In [ ]:
import sys
import os
import json
import yaml
import warnings
from pathlib import Path
from datetime import datetime
from getpass import getpass

import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

print("Libraries imported")

## Part 2: GitHub Configuration

In [ ]:
# Configure these with your GitHub details
GITHUB_USERNAME = input("Enter your GitHub username: ")
GITHUB_EMAIL = input("Enter your GitHub email: ")
GITHUB_REPO = "financial-ml-system"

print(f"\nGitHub Username: {GITHUB_USERNAME}")
print(f"GitHub Email: {GITHUB_EMAIL}")
print(f"Repository: {GITHUB_REPO}")

In [ ]:
# Get GitHub token securely
GITHUB_TOKEN = getpass("Enter your GitHub Personal Access Token: ")

if len(GITHUB_TOKEN) < 20:
    print("WARNING: Token seems too short")
else:
    print(f"Token received (length: {len(GITHUB_TOKEN)})")

## Part 3: Create Project Structure

In [ ]:
PROJECT_ROOT = Path('/content/financial-ml-system')

directories = [
    'src/data',
    'src/features',
    'src/models',
    'src/signals',
    'src/backtesting',
    'src/utils',
    'notebooks',
    'data/raw',
    'data/processed',
    'models/svm',
    'models/nn',
    'results',
    'config'
]

for directory in directories:
    dir_path = PROJECT_ROOT / directory
    dir_path.mkdir(parents=True, exist_ok=True)
    
    if directory.startswith('src/'):
        (dir_path / '__init__.py').touch()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)
print(f"Created {len(directories)} directories")
print(f"Working directory: {os.getcwd()}")

## Part 4: Create Configuration Files

In [ ]:
config = {
    'data': {
        'default_ticker': 'SPY',
        'start_date': '2018-01-01',
        'train_test_split': 0.8
    },
    'features': {
        'sma_periods': [5, 20, 50],
        'rsi_period': 14,
        'bb_period': 20,
        'atr_period': 14
    },
    'models': {
        'svm': {
            'C_options': [0.1, 1, 10, 100],
            'kernel_options': ['rbf', 'poly', 'linear'],
            'cv_folds': 5
        },
        'nn': {
            'layers': [64, 32, 16],
            'dropout': [0.3, 0.2, 0.0],
            'learning_rate': 0.001,
            'batch_size': 32,
            'epochs': 100,
            'early_stopping_patience': 10
        }
    },
    'trading': {
        'regime_threshold': 0.6,
        'return_threshold_buy': 0.005,
        'return_threshold_sell': -0.005
    }
}

with open(PROJECT_ROOT / 'config' / 'config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Configuration created")

## Part 5: Create Utility Modules

In [ ]:
# constants.py
constants = '''from pathlib import Path

PROJECT_ROOT = Path('/content/financial-ml-system')
DATA_DIR = PROJECT_ROOT / 'data'
MODELS_DIR = PROJECT_ROOT / 'models'
RESULTS_DIR = PROJECT_ROOT / 'results'
CONFIG_DIR = PROJECT_ROOT / 'config'

REGIME_BEAR = 0
REGIME_NEUTRAL = 1
REGIME_BULL = 2

REGIME_NAMES = {
    REGIME_BEAR: 'Bear',
    REGIME_NEUTRAL: 'Neutral',
    REGIME_BULL: 'Bull'
}

SIGNAL_BUY = 'BUY'
SIGNAL_SELL = 'SELL'
SIGNAL_HOLD = 'HOLD'

TRADING_DAYS_PER_YEAR = 252
RISK_FREE_RATE = 0.02
'''

with open(PROJECT_ROOT / 'src' / 'utils' / 'constants.py', 'w') as f:
    f.write(constants)

print("Created: src/utils/constants.py")

In [ ]:
# config_loader.py
config_loader = '''import yaml
from pathlib import Path
from typing import Any

class Config:
    def __init__(self, config_path: str = '/content/financial-ml-system/config/config.yaml'):
        self.config_path = Path(config_path)
        self._config = self._load_config()
    
    def _load_config(self):
        with open(self.config_path, 'r') as f:
            return yaml.safe_load(f)
    
    def get(self, key: str, default: Any = None) -> Any:
        keys = key.split('.')
        value = self._config
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        return value

config = Config()
'''

with open(PROJECT_ROOT / 'src' / 'utils' / 'config_loader.py', 'w') as f:
    f.write(config_loader)

print("Created: src/utils/config_loader.py")

In [ ]:
# helpers.py
helpers = '''import numpy as np
import pandas as pd
from typing import Tuple

def calculate_returns(prices: pd.Series, periods: int = 1) -> pd.Series:
    return prices.pct_change(periods=periods)

def calculate_sharpe_ratio(returns: pd.Series, risk_free_rate: float = 0.02, 
                          periods_per_year: int = 252) -> float:
    excess_returns = returns - risk_free_rate / periods_per_year
    return np.sqrt(periods_per_year) * excess_returns.mean() / excess_returns.std()

def calculate_max_drawdown(prices: pd.Series) -> float:
    cumulative = (1 + calculate_returns(prices)).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

def time_series_split(data: pd.DataFrame, test_size: float = 0.2) -> Tuple[pd.DataFrame, pd.DataFrame]:
    split_idx = int(len(data) * (1 - test_size))
    return data.iloc[:split_idx], data.iloc[split_idx:]
'''

with open(PROJECT_ROOT / 'src' / 'utils' / 'helpers.py', 'w') as f:
    f.write(helpers)

print("Created: src/utils/helpers.py")

## Part 6: Create README and Requirements

In [ ]:
readme = '''# Financial ML System

Institutional-grade market prediction system combining SVM and Neural Networks.

## Notebooks

Run in sequence:
1. `01_data_pipeline.ipynb` - Data ingestion and processing
2. `02_feature_engineering.ipynb` - Technical indicators
3. `03_svm_training.ipynb` - Market regime classification
4. `04_nn_training.ipynb` - Return prediction
5. `05_signal_generation.ipynb` - Trading signals
6. `06_backtesting.ipynb` - Performance analysis

## Quick Start in Colab

```python
!git clone https://github.com/''' + GITHUB_USERNAME + '''/financial-ml-system.git
%cd financial-ml-system
!pip install -r requirements.txt
```

Then open any notebook from the `notebooks/` directory.
'''

with open(PROJECT_ROOT / 'README.md', 'w') as f:
    f.write(readme)

print("Created: README.md")

In [ ]:
requirements = '''numpy>=1.24.0
pandas>=2.0.0
scikit-learn>=1.3.0
tensorflow>=2.13.0
yfinance>=0.2.28
matplotlib>=3.7.0
seaborn>=0.12.0
plotly>=5.15.0
pyyaml>=6.0
joblib>=1.3.0
'''

with open(PROJECT_ROOT / 'requirements.txt', 'w') as f:
    f.write(requirements)

print("Created: requirements.txt")

## Part 7: Initialize Git and Push to GitHub

In [ ]:
# Configure git
!git config --global user.email "{GITHUB_EMAIL}"
!git config --global user.name "{GITHUB_USERNAME}"

print("Git configured")

In [ ]:
# Create .gitignore
gitignore = '''__pycache__/
*.py[cod]
*.ipynb_checkpoints
data/raw/*
data/processed/*
models/*
results/*
!data/raw/.gitkeep
!data/processed/.gitkeep
!models/.gitkeep
!results/.gitkeep
.DS_Store
'''

with open('.gitignore', 'w') as f:
    f.write(gitignore)

# Create .gitkeep files
for dir_path in ['data/raw', 'data/processed', 'models', 'results']:
    (PROJECT_ROOT / dir_path / '.gitkeep').touch()

print(".gitignore created")

In [ ]:
# Initialize repository
!git init
!git branch -M main
!git add .
!git commit -m "Initial commit: Project setup"

print("Git repository initialized")

In [ ]:
# Add remote and push
repo_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{GITHUB_REPO}.git"

!git remote add origin {repo_url}
!git push -u origin main

print(f"\nPushed to GitHub successfully!")
print(f"View at: https://github.com/{GITHUB_USERNAME}/{GITHUB_REPO}")

## Part 8: Validation

In [ ]:
print("Validation Results")
print("=" * 50)
print(f"Project Directory: {PROJECT_ROOT.exists()}")
print(f"Config File: {(PROJECT_ROOT / 'config' / 'config.yaml').exists()}")
print(f"Utils Module: {(PROJECT_ROOT / 'src' / 'utils' / 'constants.py').exists()}")
print(f"README: {(PROJECT_ROOT / 'README.md').exists()}")
print(f"Requirements: {(PROJECT_ROOT / 'requirements.txt').exists()}")
print("=" * 50)
print("\nSetup complete!")

## Next Steps

Your project is now on GitHub!

### Access notebooks:
1. Go to: https://colab.research.google.com
2. File → Open notebook → GitHub tab
3. Enter: `your-username/financial-ml-system`
4. Select any notebook

### Or use direct URL:
```
https://colab.research.google.com/github/YOUR-USERNAME/financial-ml-system/blob/main/notebooks/01_data_pipeline.ipynb
```

### To update later:
```python
!git add .
!git commit -m "Update"
!git push origin main
```